## **Inference for Finetuned LeoLM**

Run all the cells below to generate a scienticially rewriten version of your input text. 
Please read carefully all the instructions in the markdowns!

In [1]:
%%capture
!pip install accelerate bitsandbytes transformers trl

In [2]:
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline,
    logging as hf_logging,
)

/opt/conda/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warn(f"Failed to load image Python extension: {e}")


If you want to use an updated model, please visit my [Hugging Face Profile](https://huggingface.co/JannesKl) and
copy the adress of the latest Model called 'JannesKl/DocumentStyleCheckerV...' and assign it to the variable 'model_name' below.

In [3]:
# Ignore warnings
hf_logging.set_verbosity_error()

# Load the model and tokenizer
model_name = "JannesKl/DocumentStyleCheckerV1" 
new_model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.81k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/949 [00:00<?, ?B/s]

In [4]:
#Loading Pipeline
pipe = pipeline(task="text-generation", model=new_model, tokenizer=tokenizer, max_length = 500)

In [5]:
def extract_text(generated_text):
    # Find the position of the first occurrence of " [/INST]"
    first_inst_pos = generated_text.find(" [/INST]")
    if first_inst_pos == -1:
        return ""  # Return empty string if " [/INST]" is not found
    
    # Try to find the second occurrence of " [/INST]"
    second_inst_pos = generated_text.find(" [/INST]", first_inst_pos + len(" [/INST]"))
    
    if second_inst_pos == -1:
        # If the second " [/INST]" is not found, return the text from the first " [/INST]" to the end
        start_pos = first_inst_pos + len(" [/INST]")
        extracted_text = generated_text[start_pos:].strip()
    else:
        # If the second " [/INST]" is found, return the text between the first and second " [/INST]"
        start_pos = first_inst_pos + len(" [/INST]")
        extracted_text = generated_text[start_pos:second_inst_pos].strip()
    
    return extracted_text

In [6]:
def rewrite_scientifically(user_input):
    user_input = user_input
    system_prompt = "Bitte formuliere den folgenden Text in eine wissenschaftliche Schreibweise um:"

    #prompt instruction: https://llama.meta.com/docs/how-to-guides/prompting/

    B_INST, E_INST = "<s>[INST]", "[/INST]"
    B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
    
    prompt = f"{B_INST} {B_SYS}{system_prompt}{E_SYS}{user_input} {E_INST}"

    result = pipe(prompt)

    generated_text = result[0]['generated_text']

    return generated_text #extract_text(generated_text) instead, if you want only the answer, but its not 100% reliable

Please insert your input string between the "" before you run the cell.

In [1]:
rewrite_scientifically("")

NameError: name 'rewrite_scientifically' is not defined

## **Examples: DucomentStyleCheckerV2**:

To test the finetuned modell, i created with ChatGPT a list of thirty unscientific sentences in the topics computer science, physics and chemistry: 

__Informatik:__

1. "Rechnerarchitekturen sind die verschiedenen Designs, die Computer verwenden, um Daten herumzuschieben."
2. "Komplexitätsklassen zeigen einfach an, wie schnell ein Computer ein Problem lösen kann."
3. "Ein Betriebssystem ist wie ein großer Chef, der entscheidet, welche Programme wann laufen."
4. "Formale Sprachen sind spezielle Arten zu sprechen, die nur Computer verstehen können."
5. "Computer benutzen Bits und Bytes, um Sachen zu rechnen und zu speichern."

__Kryptographie:__

1. "Chiffren sind wie Geheimsprachen, die niemand außer den Eingeweihten verstehen kann."
2. "Verschlüsselung bedeutet, Nachrichten in unleserliche Buchstabensalat zu verwandeln."
3. "Kryptographie ist einfach das Schreiben von Geheimcodes, damit keiner die Nachrichten lesen kann."
4. "Ein Schlüssel in der Kryptographie ist wie ein Passwort, das die Geheimschrift lesbar macht."
5. "Verschlüsselte Nachrichten sind wie Briefe in einem verschlossenen Safe."

__Strahlungsphysik:__

1. "Das Zerfallsgesetz beschreibt, wie schnell radioaktive Sachen kaputt gehen."
2. "Dosisgrößen sind nur komplizierte Begriffe dafür, wie viel Strahlung man abbekommt."
3. "Strahlung ist einfach unsichtbares Licht, das manchmal gefährlich ist."
4. "Strahlenschutz bedeutet, sich vor unsichtbaren Strahlen zu verstecken."
5. "Radioaktive Strahlung ist wie kleine unsichtbare Teilchen, die durch alles hindurchgehen können."

__Chemie der Biologie:__

1. "Das Periodensystem ist eine große Tabelle, die alle Arten von Atomen zeigt."
2. "Derivatisierung ist, wenn Chemiker Moleküle verändern, um sie besser zu untersuchen."
3. "Kohlenstoff ist besonders, weil er gerne mit anderen Atomen Freundschaften schließt."
4. "Biochemie ist einfach Chemie, die in Lebewesen passiert."
5. "Kohlenstoffverbindungen sind wie Lego-Steine, die alle möglichen Formen bilden können."